In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import yfinance as yf
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [77]:
msft=yf.download('msft',start='2019-01-01')

[*********************100%***********************]  1 of 1 completed


In [78]:
msft

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,99.550003,101.750000,98.940002,101.120003,96.421875,35329300
2019-01-03,100.099998,100.190002,97.199997,97.400002,92.874718,42579100
2019-01-04,99.720001,102.510002,98.930000,101.930000,97.194237,44060600
2019-01-07,101.639999,103.269997,100.980003,102.059998,97.318192,35656100
2019-01-08,103.040001,103.970001,101.709999,102.800003,98.023834,31514400
...,...,...,...,...,...,...
2023-06-07,331.649994,334.489990,322.500000,323.380005,323.380005,40717100
2023-06-08,323.940002,326.640015,323.350006,325.260010,325.260010,23277700
2023-06-09,324.989990,329.989990,324.410004,326.790009,326.790009,22514900


In [79]:
def get_adx(High,Low,Close,lookback):
    plus_dm = High.diff()
    minus_dm = Low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    tr1 = pd.DataFrame(High - Low)
    tr2 = pd.DataFrame(abs(High - Close.shift(1)))
    tr3 = pd.DataFrame(abs(Low - Close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()
    return plus_di, minus_di, adx_smooth



In [80]:
msft['plus_di'] = pd.DataFrame(get_adx(msft['High'], msft['Low'], msft['Close'], 14)[0]).rename(columns = {0:'plus_di'})
msft['minus_di'] = pd.DataFrame(get_adx(msft['High'], msft['Low'],msft['Close'], 14)[1]).rename(columns = {0:'minus_di'})
msft['adx'] = pd.DataFrame(get_adx(msft['High'], msft['Low'], msft['Close'], 14)[2]).rename(columns = {0:'adx'})
msft = msft.dropna()
msft.tail()

,Open,High,Low,Close,Adj Close,Volume,plus_di,minus_di,adx
Date,,,,,,,,,
2023-06-07,331.649994,334.489990,322.500000,323.380005,323.380005,40717100,29.251494,22.970660,46.719671
2023-06-08,323.940002,326.640015,323.350006,325.260010,325.260010,23277700,27.765176,21.803482,44.241633
2023-06-09,324.989990,329.989990,324.410004,326.790009,326.790009,22514900,28.450192,19.561454,41.973693
2023-06-12,328.579987,332.100006,325.160004,331.850006,331.850006,24260300,27.950715,17.722225,40.317783
2023-06-13,334.464996,336.980011,330.399994,331.970001,331.970001,10779381,31.272647,16.608755,39.079582


In [81]:
def get_rsi(Close, lookback):
    ret = Close.diff()
    up = []
    down = []
    
    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)
    
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    
    up_ewm = up_series.ewm(com = lookback - 1, adjust = False).mean()
    down_ewm = down_series.ewm(com = lookback - 1, adjust = False).mean()
    
    rs = up_ewm/down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns = {0:'rsi'}).set_index(Close.index)
    rsi_df = rsi_df.dropna()
    
    return rsi_df[3:]

msft['rsi_14'] = get_rsi(msft['Close'], 14)
msft = msft.dropna()
msft.tail()

/var/folders/4x/zccyl8cx4sz3k2bfp2dszw5r0000gn/T/ipykernel_15359/659328882.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msft['rsi_14'] = get_rsi(msft['Close'], 14)


,Open,High,Low,Close,Adj Close,Volume,plus_di,minus_di,adx,rsi_14
Date,,,,,,,,,,
2023-06-07,331.649994,334.489990,322.500000,323.380005,323.380005,40717100,29.251494,22.970660,46.719671,55.515915
2023-06-08,323.940002,326.640015,323.350006,325.260010,325.260010,23277700,27.765176,21.803482,44.241633,57.129000
2023-06-09,324.989990,329.989990,324.410004,326.790009,326.790009,22514900,28.450192,19.561454,41.973693,58.449522
2023-06-12,328.579987,332.100006,325.160004,331.850006,331.850006,24260300,27.950715,17.722225,40.317783,62.557181
2023-06-13,334.464996,336.980011,330.399994,331.970001,331.970001,10779381,31.272647,16.608755,39.079582,62.651476


In [82]:
def adx_rsi_strategy(prices, adx, pdi, ndi, rsi):
    buy_price = []
    sell_price = []
    adx_rsi_signal = []
    signal = 0
    
    for i in range(len(prices)):
        if adx[i] > 35 and pdi[i] < ndi[i] and rsi[i] < 50:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                adx_rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                adx_rsi_signal.append(0)
                
        elif adx[i] > 35 and pdi[i] > ndi[i] and rsi[i] > 50:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                adx_rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                adx_rsi_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            adx_rsi_signal.append(0)
                        
    return buy_price, sell_price, adx_rsi_signal

buy_price, sell_price, adx_rsi_signal = adx_rsi_strategy(msft['Close'], msft['adx'], msft['plus_di'], msft['minus_di'], msft['rsi_14'])

In [83]:
position = []
for i in range(len(adx_rsi_signal)):
    if adx_rsi_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)
        
for i in range(len(msft['Close'])):
    if adx_rsi_signal[i] == 1:
        position[i] = 1
    elif adx_rsi_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]
        
adx = msft['adx']
pdi = msft['plus_di']
ndi = msft['minus_di']
rsi = msft['rsi_14'] 
Close_price = msft['Close']
adx_rsi_signal = pd.DataFrame(adx_rsi_signal).rename(columns = {0:'adx_rsi_signal'}).set_index(msft.index)
position = pd.DataFrame(position).rename(columns = {0:'adx_rsi_position'}).set_index(msft.index)

frames = [Close_price, adx, pdi, ndi, rsi, adx_rsi_signal, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

strategy

,Close,adx,plus_di,minus_di,rsi_14,adx_rsi_signal,adx_rsi_position
Date,,,,,,,
2019-01-29,102.940002,30.519958,23.891741,28.472539,15.705387,0,1
2019-01-30,106.379997,26.122549,26.781205,24.632146,50.287606,0,1
2019-01-31,104.430000,22.268891,23.306323,26.107988,40.215740,0,1
2019-02-01,102.779999,19.691615,21.117035,26.945680,34.008826,0,1
2019-02-04,105.739998,18.481490,24.754410,23.508060,49.166205,0,1
...,...,...,...,...,...,...,...
2023-06-07,323.380005,46.719671,29.251494,22.970660,55.515915,0,0
2023-06-08,325.260010,44.241633,27.765176,21.803482,57.129000,0,0
2023-06-09,326.790009,41.973693,28.450192,19.561454,58.449522,0,0


In [84]:
msft_ret = pd.DataFrame(np.diff(msft['Close'])).rename(columns = {0:'returns'})
adx_rsi_strategy_ret = []

for i in range(len(msft_ret)):
    returns = msft_ret['returns'][i]*strategy['adx_rsi_position'][i]
    adx_rsi_strategy_ret.append(returns)
    
adx_rsi_strategy_ret_df = pd.DataFrame(adx_rsi_strategy_ret).rename(columns = {0:'adx_rsi_returns'})
investment_value = 100000
number_of_stocks = floor(investment_value/msft['Close'][0])
adx_rsi_investment_ret = []

for i in range(len(adx_rsi_strategy_ret_df['adx_rsi_returns'])):
    returns = number_of_stocks*adx_rsi_strategy_ret_df['adx_rsi_returns'][i]
    adx_rsi_investment_ret.append(returns)

adx_rsi_investment_ret_df = pd.DataFrame(adx_rsi_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(adx_rsi_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret/investment_value)*100)
print(cl('Profit gained from the ADX RSI strategy by investing $100k in MSFT : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the ADX RSI strategy : {}%'.format(profit_percentage), attrs = ['bold']))

Profit gained from the ADX RSI strategy by investing $100k in MSFT : 112092.24
Profit percentage of the ADX RSI strategy : 112%
